<a href="https://colab.research.google.com/github/GiuBonacchi/Music/blob/main/sistema_de_reconhecimento_musical_util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colocar dataframe com nome novos_dados.csv

In [85]:
import pickle
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

In [86]:
with open('modelo_random_forest.pkl', 'rb') as f:
    model = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('colunas_usadas_no_treino.pkl', 'rb') as f:
    colunas_treino = pickle.load(f)

In [87]:
novos_dados = pd.read_csv('musicas_teste_com_erros.csv')
print(novos_dados.head())

               music           artist duration_ms  music_popularity  \
0       Shape of You       Ed Sheeran      233712                85   
1  Bohemian Rhapsody            Queen     -354000                90   
2    Blinding Lights       The Weeknd      200500                88   
3        Billie Jean  Michael Jackson         abc                92   
4         Wonderwall            Oasis      259000                80   

   artist_popularity     followers           genres  
0               95.0      10000000              pop  
1                NaN       5000000             rock  
2               93.0       8000000              NaN  
3               97.0  desconhecido  pop,wrong_genre  
4               85.0       3000000       rock,indie  


In [88]:
music = novos_dados['music']
artist = novos_dados['artist']

In [89]:
dummies = novos_dados['genres'].str.get_dummies(sep=',')
novos_dados = pd.concat([novos_dados.drop(columns=['genres']), dummies], axis=1)

In [90]:
colunas_que_faltam = [col for col in colunas_treino if col not in novos_dados.columns]
df_falta = pd.DataFrame(0, index=novos_dados.index, columns=colunas_que_faltam)
novos_dados = pd.concat([novos_dados, df_falta], axis=1)
novos_dados = novos_dados[colunas_treino]

In [91]:
numeric_cols = ['duration_ms', 'music_popularity', 'artist_popularity', 'followers']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [92]:
#converte colunas com valores invalidos pra nan
novos_dados[numeric_cols] = novos_dados[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [93]:
novos_dados[numeric_cols] = novos_dados[numeric_cols].fillna(0)

In [94]:
novos_dados[numeric_cols] = scaler.transform(novos_dados[numeric_cols])

In [95]:
previsoes = model.predict(novos_dados)

In [96]:
print(previsoes)

[1 1 1 1 1]


In [97]:
resultado = pd.DataFrame({
    'music': music,
    'artist': artist,
    'liked_prediction': previsoes
})

In [98]:
print(resultado.head())

               music           artist  liked_prediction
0       Shape of You       Ed Sheeran                 1
1  Bohemian Rhapsody            Queen                 1
2    Blinding Lights       The Weeknd                 1
3        Billie Jean  Michael Jackson                 1
4         Wonderwall            Oasis                 1
